In [1]:
import sys
sys.path.append('../../../src/')

import os
from typing import Dict
from os import PathLike
from pathlib import Path
import csv 
import glob

from aind_vr_foraging_analysis import utils
from aind_vr_foraging_analysis.utils import parse, processing, plotting_utils as plotting, AddExtraColumns
import datetime

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages

import seaborn as sns
import pandas as pd
import numpy as np
import datetime

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Rectangle


colors = sns.color_palette()
odor_list_color = [colors[8], colors[0], colors[2], colors[4]]

from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from pathlib import Path

In [2]:

# Function to identify peaks and troughs
#neds to be modified to filter peaks when signal is bad
def identify_peaks_and_troughs(data, width_peaks=5, prominence_peaks=0.1, width_troughs=3, prominence_troughs=0.1):
    data = data.dropna()
    data = data[~data.index.duplicated(keep='first')]
    data = data.sort_index()
    data_values = data.values.squeeze()
    peaks, _ = find_peaks(data_values, width=width_peaks, prominence=prominence_peaks)
    troughs, _ = find_peaks(-data_values, width=width_troughs, prominence=prominence_troughs)
    troughs = troughs[troughs > peaks[0]]
    peak_times = data.index[peaks]
    trough_times = data.index[troughs]
    return peak_times, trough_times


In [3]:

# Function to align sniff peaks with rewards
def align_sniff_peaks_with_rewards(stream_data, reward_sites, width_peaks=5, prominence_peaks=0.1, width_troughs=3, prominence_troughs=0.1):
    sniff_aligned = reward_sites.copy()
    peak_times, trough_times = identify_peaks_and_troughs(stream_data.breathing, width_peaks, prominence_peaks, width_troughs, prominence_troughs)
    first_peaks = [np.searchsorted(peak_times, start_time) for start_time in sniff_aligned.index]
    first_sniff_peak_start_time = [peak_times[fp] for fp in first_peaks]
    sniff_aligned['first_sniff_peak'] = first_sniff_peak_start_time
    sniff_aligned = sniff_aligned.set_index('first_sniff_peak')
    sniff_aligned.index.name = 'times'
    return sniff_aligned

In [7]:

# Define the path to the mouse folder
mouse_folder_path = Path(f'/Volumes/aind/scratch/vr-foraging/data/{mouse}')

# Function to process a single session and return peaks_df
def process_session(session_path):
    # Extract mouse_id from the first 6 numbers in the file name
    mouse = session_path.name[:6]
    # Extract session from the first 8 numbers after mouse_id
    session = session_path.name[6:14]
    try:

        data = parse.load_session_data(session_path)
        reward_sites, active_site, config = parse.parse_dataframe(data)
        reward_sites = AddExtraColumns(reward_sites, active_site, run_on_init=True).reward_sites
        active_site = AddExtraColumns(reward_sites, active_site, run_on_init=False).add_time_previous_intersite_interpatch()
        color_dict_label = {}
        dict_odor = {}
        list_patches = parse.TaskSchemaProperties(data).patches
        for i, patches in enumerate(list_patches):
            color_dict_label[patches['label']] = odor_list_color[i]
            dict_odor[i] = patches['label']
        stream_data = parse.ContinuousData(data)
        encoder_data = stream_data.encoder_data

        active_site['next_intersite'] = active_site.index.to_series().shift(-1)
        reward_sites = active_site.loc[active_site['label'] == 'RewardSite']
        reward_sites['time_in_odor_site'] = reward_sites.next_intersite - reward_sites.index
        plot_df = reward_sites[['time_in_odor_site', 'odor_label', 'active_patch']]
        label_dict = {**{
            "InterSite": '#808080',
            "InterPatch": '#b3b3b3'}, **color_dict_label}
        sniff_aligned = align_sniff_peaks_with_rewards(stream_data, reward_sites)
        trial_summary_breathing = plotting.trial_collection(
            sniff_aligned[['has_choice', 'visit_number', 'odor_label', 'odor_sites', 'time_in_odor_site']], 
            stream_data.breathing, 
            mouse, 
            session, 
            window=(-2, 8), 
            taken_col='data'
        )
        
        peaks_data = []
        for odor_label in trial_summary_breathing['odor_label'].unique():
            odor_df = trial_summary_breathing[trial_summary_breathing['odor_label'] == odor_label]
            for site in odor_df['odor_sites'].unique():
                site_df = odor_df[odor_df['odor_sites'] == site]
                signal = site_df.set_index('times')['data']
                odor_end_time = odor_df['time_in_odor_site'].unique()[0]
                x_start = 0
                x_end = odor_end_time
                filtered_signal = signal[(signal.index >= x_start) & (signal.index <= x_end)]
                if filtered_signal.empty:
                    continue
                peak_times, _ = find_peaks(filtered_signal, width=5, prominence=0.1)
                if len(peak_times) > 0:
                    peak_times_indices = filtered_signal.index[peak_times]
                    peak_count = len(peak_times_indices)
                    epoch_duration = x_end - x_start
                    peak_frequency = peak_count / epoch_duration if epoch_duration > 0 else 0
                    peaks_data.append({
                        'session': session_path.name,
                        'odor_label': odor_label, 
                        'odor_site': site, 
                        'peak_count': peak_count,
                        'peak_frequency': peak_frequency,
                        'epoch_duration': epoch_duration
                    })
                    peak_count_after_1s = sum(peak <= 1 for peak in peak_times_indices)
                    peaks_data[-1]['peak_count_after_1s'] = peak_count_after_1s

        return pd.DataFrame(peaks_data)
    except Exception as e:
        print(f"Error processing session {session_path.name}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

# Iterate through all sessions and aggregate results
all_peaks_data = []
for session_dir in os.listdir(mouse_folder_path):
    session_path = mouse_folder_path / session_dir
    if session_path.is_dir():
        session_peaks_df = process_session(session_path)
        if not session_peaks_df.empty:
            all_peaks_data.append(session_peaks_df)

# concatenate all session data into a single DataFrame
if all_peaks_data:
    summary_peaks_df = pd.concat(all_peaks_data, ignore_index=True)
else:
    summary_peaks_df = pd.DataFrame()  # Handle the case where no data was processed

# Print the summary DataFrame
print(summary_peaks_df)


Error processing session 717716_20240711T104507: 'odor_label'
Error processing session 717716_20240712T092200: "None of ['Seconds'] are in the columns"
                    session odor_label  odor_site  peak_count  peak_frequency  \
0    717716_20240710T110945     ODOR_B        0.0          22        6.929186   
1    717716_20240710T110945       NULL        1.0          37        1.503392   
2    717716_20240710T110945       NULL        2.0          32        1.300231   
3    717716_20240710T110945       NULL        3.0          50        2.031611   
4    717716_20240712T093131       NULL        0.0          41        0.291431   
..                      ...        ...        ...         ...             ...   
859  717716_20240803T105420     ODOR_C       90.0          12        7.782020   
860  717716_20240803T105420     ODOR_C       94.0          13        8.430522   
861  717716_20240803T105420     ODOR_C       97.0          10        6.485017   
862  717716_20240803T105420     ODOR_C

In [8]:
#save df to csv in /Users/nehal.ajmal/Documents/aindproject/results
summary_peaks_df.to_csv(f'/Users/nehal.ajmal/Documents/aindproject/results/{mouse}_peaks_data.csv', index=False)

In [ ]:
#average peak count after 1s for each odor label
average_peak_count_after_1s = summary_peaks_df.groupby('odor_label')['peak_count_after_1s'].mean()
print(average_peak_count_after_1s)

odor_label
NULL      7.217949
ODOR_A    8.192771
ODOR_B    7.166667
ODOR_C    7.055556
Name: peak_count_after_1s, dtype: float64
